# 데이터 마트 구축하기
+ Spark로 전처리한 열 지향 스토리지 활용 고속 집계
+ 상세페이지: 237~
+ 출처: 빅데이터를 지탱하는 기술 (나시다 케이스케)

## 1. 데이터 확인

In [1]:
import os
import time 

In [12]:
spark.table("ko_words_20220214").count()

11629522

In [15]:
query = """
select 
    substr(time, 1, 13) as time
    , word
    , count(*) as cnt
from ko_words_20220214
group by time, word
order by time, cnt desc, word
"""

spark.sql(query).show(10)
spark.sql(query).count()

22/02/16 23:01:08 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
22/02/16 23:01:08 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
22/02/16 23:01:08 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
22/02/16 23:01:08 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
22/02/16 23:01:08 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
22/02/16 23:01:08 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
22/02/16 23:01:08 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
22/02/16 23:01:08 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
22/02/16 23:01:10 WARN RowBasedKeyValueBatch: Calling spill() on

+-------------+------------+---+
|         time|        word|cnt|
+-------------+------------+---+
|2022-02-06 12|           :| 13|
|2022-02-06 12|           :| 13|
|2022-02-06 12|         kkk| 11|
|2022-02-06 12|           o| 11|
|2022-02-06 12|      ㅋㅋㅋ| 11|
|2022-02-06 12|          𐨛| 11|
|2022-02-06 12|           X| 10|
|2022-02-06 12|           :|  9|
|2022-02-06 12|           o|  9|
|2022-02-06 12|스타킹페티쉬|  9|
+-------------+------------+---+
only showing top 10 rows



10987297

## 2. 등록 횟수가 적은 단어의 수를 조사

In [17]:
query = """
select
    t.cnt
    , count(*) as words_cnt
from (
    select
        word
        , count(*) as cnt 
    from ko_words_20220214
    group by word
) t
group by t.cnt
order by words_cnt desc
"""

spark.sql(query).show(10)

+---+---------+
|cnt|words_cnt|
+---+---------+
|  1|  1392445|
|  2|   213171|
|  3|    92150|
|  4|    52327|
|  5|    34146|
|  6|    25165|
|  7|    18324|
|  8|    14307|
|  9|    11471|
| 10|     9661|
+---+---------+
only showing top 10 rows



## 3. 단어를 카테고리로 나누는 디멘전 테이블 생성

In [25]:
query = """
select 
    word
    , count
    , if(count > 1000, word, concat('count=', count)) as category
from (
    select
        word
        , count(*) as count
    from ko_words_20220214
    group by word
) t
-- where count < 5000 주석도 OK
order by count desc

"""
spark.sql(query).show(10)

+----+------+--------+
|word| count|category|
+----+------+--------+
|  RT|561390|      RT|
|너무| 39247|    너무|
|진짜| 36304|    진짜|
|  이| 24138|      이|
|   :| 23896|       :|
|  다| 23381|      다|
|  수| 23294|      수|
|   -| 21314|       -|
|  아| 19835|      아|
|  한| 19593|      한|
+----+------+--------+
only showing top 10 rows



## 4. 임시 뷰로 등록

In [26]:
spark.sql(query).createOrReplaceTempView("word_category")

In [38]:
# 추가 정리: csv로 저장하기
df_word_category = spark.sql("select * from word_category").toPandas()

In [39]:
df_word_category.to_csv("word_category.csv", index=False, encoding="utf8")

In [30]:
spark.sql("select * from word_category").show(2)

+----+------+--------+
|word| count|category|
+----+------+--------+
|  RT|561390|      RT|
|너무| 39247|    너무|
+----+------+--------+
only showing top 2 rows



## 5. 집계
+ 누적 count 1000회 미만의 중요하지 않은 단어는 제외해서 집계
+ count 기준을 잡기 위해 windows function 사용 예정 
+ https://spark.apache.org/docs/latest/sql-ref-syntax-qry-select-window.html

In [34]:
query = """
select
    substr(a.time, 1, 13) time
    , b.category
    , count(*) as count
from ko_words_20220214 a
left join word_category b
on a.word = b.word
group by 1, 2
"""
spark.sql(query).count()
spark.sql(query).show(10)

22/02/16 23:15:53 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
22/02/16 23:15:53 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
22/02/16 23:15:53 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
22/02/16 23:15:53 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
22/02/16 23:15:54 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
22/02/16 23:15:54 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
22/02/16 23:15:54 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
22/02/16 23:15:54 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
22/02/16 23:16:00 WARN RowBasedKeyValueBatch: Calling spill() on

+-------------+---------+-----+
|         time| category|count|
+-------------+---------+-----+
|2022-02-11 00|  count=3| 1296|
|2022-02-12 18| count=29|  129|
|2022-02-10 08| count=53|  211|
|2022-02-12 22| count=16|  104|
|2022-02-10 13| count=22|  373|
|2022-02-13 06|  count=7|  962|
|2022-02-06 16|  count=4| 1620|
|2022-02-10 14| count=12|  855|
|2022-02-14 10|count=864|    2|
|2022-02-14 11|   #ATEEZ|   18|
+-------------+---------+-----+
only showing top 10 rows

